1. Check later append properties is not embedding
2. Implement deduplication and +hash and +topic list simultaneously
3. Upgrade frontend to use topic list 

In [ ]:
from dotenv import load_dotenv
import os
import weaviate

load_dotenv()
import sys

# add path to sys
sys.path.append("/workspaces/askem/askem/retriever")

In [ ]:
client = weaviate.Client(
    url=os.getenv("WEAVIATE_URL"),
    auth_client_secret=weaviate.AuthApiKey(api_key=os.getenv("WEAVIATE_APIKEY")),
)

In [ ]:
client.schema.delete_all()

In [ ]:
from askem.retriever.base import init_retriever

init_retriever(client)

In [ ]:
from askem.retriever.migrate import convert_data, MigrationManager

In [ ]:
client.schema.get()

In [ ]:
client.query.aggregate("Passage").with_meta_count().do()

In [ ]:
q = (
    client.query.get("Passage", ["topic", "topic_list", "text_hash"])
    .with_additional(["id"])
    .with_limit(1)
)
y = q.do()

In [ ]:
from base import get_schema

In [ ]:
schema = get_schema()

In [ ]:
schema

In [ ]:
client.schema.update_config("PassageV2", schema)

In [ ]:
y

In [ ]:
from askem.utils import import_documents


import_documents(
    client,
    input_dir="data/debug_data",
    class_name="Passage",
    topic="covid",
    doc_type="paragraph",
    duplicate_check=False,
)